In [ ]:
%load_ext autoreload
%autoreload 2
from IPython import get_ipython; ipython = get_ipython()
#%run -i ../exec_HPC.py 0
import os 
import sys
from os.path import join as pjoin
p = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'state-space-adaptation' )
sys.path.append(p)
p2 = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'state-space-adaptation','state_space_fit_toolbox' )
sys.path.append(p2)
import pandas as pd

df = pd.read_pickle('/home/demitau/ownCloud/Current/output/memerr/workstation_CRNL/row=trial.pkl.zip')

import state_space_Tan2014 as tan
from state_space_Tan2014 import *

scripts_dir = pjoin(p, 'state_space_fit_toolbox')
scr = pjoin(scripts_dir, 'run_example.py')

assert not df.duplicated(['subject','trial_index']).any()
subjects = df['subject'].unique()

dfos = df.query('subject == @subjects[0]')

assert 0 == len( dfos.loc[dfos['perturbation'].isna()] )

import pyEMtoolbox.emtb as pyEM

import pymatreader as pym
r = pym.read_mat( pjoin(scripts_dir,'example_behavior.mat') )
r.keys()
measured_error = r['measured_error']
perturb        = r['perturb']

In [ ]:
#pivot
#%xmode context

In [ ]:
# # main run
# nt = 1000
# nsub = 4
# subjects_sub = subjects[0:nsub]
# numtrain = 12
# dfc = df.query('subject in @subjects_sub and trial_index < @nt and trial_index >= @numtrain').copy()
# dfc.loc[dfc.query('error_lh2_ang_deg.abs() > 60').index, 'error_lh2_ang_deg'] = np.nan
# # TODO: maybe I should invalidate too big errors, they are clearly outliers
# coln_error = 'error_lh2_ang_deg'
# n_jobs = 10
# do_plot = 0

In [ ]:
np.isnan(measured_error).any()

In [ ]:
measured_error.shape, perturb.shape

In [ ]:
params_init_Diedrischen.shape

In [ ]:
params_Diedrischen.shape, params_init_Diedrischen.shape

In [ ]:
ipython.run_line_magic('run',' -i ' + scr)

In [ ]:

#measured_error = np.random.uniform(size=(nt,))
linalg_error_handling = 'ignore'
import traceback
try:
    # Fit a two-state model with retention factor using stochastic EM approach
    # (Albert, 2018) optimizing parameters in log-space
    r = state_space_fit(behavior_cursubj, perturb_cursubj,
            params_init_Albert, search_space_Albert, 'em', 'log',
                        linalg_error_handling = linalg_error_handling)

except Exception as e:
    #print('EXC',e)
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    ei = exc_info[2]

    #display(ei.tb_frame)
    print('\n'.join( traceback.format_exc().split('\n')[-9:]) ) 
    psf = ei
    
    lfprev,lf = None, None
    lfs = []
    cofns = []
    while psf.tb_frame is not None:
        stackframe = psf.tb_frame
        cofn = stackframe.f_code.co_filename
        #print(cofn)
        isco = cofn.find('conda') >= 0
        isaf = cofn.find('__array_function__') >= 0
        lfprev = lf
        lf = stackframe.f_locals
        if not (isco or isaf):
            display(stackframe)                        
            lfs += [lf]
            cofns += [cofn]
        psf = psf.tb_next    
        if psf is None:
            break

    print('EXC',e)

In [ ]:
#nt = 80
#measured_error = np.random.uniform(size=(nt,))
linalg_error_handling = 'ignore'
import traceback
try:
    print( params_init_Diedrischen, search_space_Diedrischen )
    r = state_space_fit(behavior_cursubj, perturb_cursubj,
            params_init_Diedrischen, search_space_Diedrischen,
            'lmse', 'norm', linalg_error_handling=linalg_error_handling)
    output, params, asymptote, noises_var, states, performances  = r


except Exception as e:
    #print('EXC',e)
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    ei = exc_info[2]

    #display(ei.tb_frame)
    print('\n'.join( traceback.format_exc().split('\n')[-6:]) ) 
    psf = ei
    
    lfprev,lf = None, None
    lfs = []
    cofns = []
    while psf.tb_frame is not None:
        stackframe = psf.tb_frame
        cofn = stackframe.f_code.co_filename
        #print(cofn)
        isco = cofn.find('conda') >= 0
        isaf = cofn.find('__array_function__') >= 0
        lfprev = lf
        lf = stackframe.f_locals
        if not (isco or isaf):
            display(stackframe)                        
            lfs += [lf]
            cofns += [cofn]
        psf = psf.tb_next    
        if psf is None:
            break

    print('EXC',e)

In [ ]:
lfind = -1
lfc  = lfs[lfind]
print(cofns[lfind], lfc.keys())

In [ ]:
locals().update(lfc)

In [ ]:
 np.mean(perturb)

In [ ]:
b[:, np.newaxis]

In [ ]:
b[:, np.newaxis] @ np.mean(perturb)

In [ ]:
params_search_ranges[:, 0::2].flatten(), params_search_ranges[:, 1::2].flatten()

In [ ]:
[0.0001, 10000], [0.0001, 10000], [0.0000001, 10]

In [ ]:
bds0 = np.vstack((params_search_ranges[:, [0, 2, 4]].flatten(), 
                 params_search_ranges[:, [1, 3, 5]].flatten() ) ).T
bds_add = np.array([[0.0001, 10000], [0.0001, 10000], [0.0000001, 10]])
print(bds0.shape, bds_add.shape)
#print(bds.T.shape)
bds = np.vstack([bds0,bds_add])
print(bds.shape)

In [ ]:
#bds should be nparamx x 2

In [ ]:
bds = list(zip(*bds.T))

In [ ]:
b_con, b_con.shape

In [ ]:
A_con.shape[0:1], A_con.shape

In [ ]:
np.broadcast_to?

In [ ]:
np.broadcast_to(b_con, (2,))

In [ ]:
from scipy.optimize import LinearConstraint
LinearConstraint(A_con, b_con, b_con)

In [ ]:
from pyEMtoolbox.emtb import generalized_expectation_maximization
#%debug
S_opt, likelihoods = generalized_expectation_maximization(
    np.hstack((params_init.flatten(), 2, 2, 5)),  # Initial guess
    behavior, perturb, np.zeros(len(behavior)), 
    np.nan * np.ones(len(behavior)), np.ones(nbStates),  # Paradigm
    bds,  # Search-space
    A_con, b_con[:,0],  # constraints
    100, 0, search_method)


In [ ]:
S_opt

In [ ]:
output_Diedrischen[:,s] = r[0]

In [ ]:
params_Diedrischen.shape

In [ ]:
r[1].shape

In [ ]:
r[3].shape

In [ ]:
params_Diedrischen[:,s] = r[1]

In [ ]:
    (output_Diedrischen[:,s], params_Diedrischen[:,s],
     asymptote_Diedrischen[s], noise_Diedrischen[s], _,
     perf_Diedrischen[:,s]) = ra
    


In [ ]:
r

In [ ]:
# @ in python means matrix multiply

In [ ]:
nbStates,2*nbStates

In [ ]:
S_opt

In [ ]:
search_method

In [ ]:
np.diag(A), b #- b[:, np.newaxis]

In [ ]:
AAarg = np.eye(nbStates) -\
        (np.diag(A) - b[:, np.newaxis] @ \
         np.ones((1, nbStates)))
print(AAarg)
AA = np.linalg.inv(AAarg)

In [ ]:
from scipy.linalg import LinAlgError as lae

In [ ]:
    asymptote = np.ones((1, nbStates)) @ AA @ (b[:, np.newaxis] @ np.mean(perturb))
    params = np.vstack((A, b, x0))


In [ ]:
nbStates

In [ ]:
params_search_ranges

In [ ]:
params_init

In [ ]:
params init shape defines num states

In [ ]:
params_search_ranges.shape

In [ ]:
bds

In [ ]:
params_search_ranges.shape

In [ ]:
params_search_ranges[0,::2]

In [ ]:
print(params_init.flatten())
bds = list(zip(params_search_ranges[0, ::2],
                    params_search_ranges[0, ::2]))
print('bds = ',bds)
res = minimize(deterministic_ss_mse, params_init.flatten(),
    args=(perturb, behavior, search_method),
    bounds=bds)


In [ ]:
traceback.format_exc()

In [ ]:
stackframe.f_code.

In [ ]:
print(params_init.flatten())
print(list(zip(params_search_ranges[:, 0],
                    params_search_ranges[:, 1])) )
res = minimize(deterministic_ss_mse, params_init.flatten(),
    args=(perturb, behavior, search_method),
    bounds=list(zip(params_search_ranges[:, 0],
                    params_search_ranges[:, 1])))


In [ ]:
for subj in subj2out_Tan:
    error = dfc.query('subject == @subj')[coln_error].to_numpy()
    perturb = dfc.query('subject == @subj')['perturbation'].to_numpy()    
    out = subj2out_Tan[subj ]
    adaptationRate_Tan = out['adaptation_rate']
    output_Tan = out['y_pred']
    plt.figure(figsize=(15,3)) # Use matplotlib.pyplot to plot figures
    plt.title('Subject ' + s) # Use string concatenation to make titles
    plt.plot(error, linewidth=2, label='Observed error') # Use label argument to add legends
    plt.plot(output_Tan, linewidth=2, label='Model prediction',alpha=0.8)
    plt.ylabel('Angle (°)')
    plt.xlabel('Trials')
    plt.plot(perturb, label='pert', color='red', ls=':')
    plt.legend() # Call legend function to show legends
    #plt.twinx() # Use twinx function to create a second y-axis
    plt.plot(30 + 50*adaptationRate_Tan, linewidth=1, label='Adaptation rate', color='green') # Use color argument to change the line color
    plt.legend() # Call legend function to show legends
    plt.ylim(-40,60)
    plt.grid(True)
    plt.show() # Call show function to display the figure


In [ ]:
for subj in subj2out_Tan: 
    print('     ',subj)
    for k,v in subj2out_Tan[subj ].items():
        if not isinstance(v,np.ndarray):
            if isinstance(v,dict):
                print(k); display(v)
            else:
                print(k,v)